In [95]:
from pathlib import Path
import re

In [96]:
FILEPATHS = [Path("../Start.py")] + list(Path("..").glob("pages/*.py"))
ALLVPATH = Path("allversion.po")
RENPATH = Path("allversion_renamed.po")

In [97]:
# Define the file abbreviations
abbr = {
    'Start.py' : "STR",
    'conclusion.py': "CON",
    'model_explain.py': "MDL",
    '_sidebar.py': "SDE",
    'first_analysis.py': "FAL",
    'method.py': "MTH",
    'contact.py': "CON",
    'photosynthesis.py': "PHO",
    'plant_memory.py': "MEM"
}

no_rep = [
    "LITERATURE",
    "LITERATURE_ONPAGE"
]

In [98]:
# Extract the text placeholders
text_re = '(?<=text\(")[\w]+(?="\))|(?<=markdown_click\(")[\w]+(?=")'
texts = {}

for FILEPATH in FILEPATHS:
    keys=[]
    with open(FILEPATH, "r") as f:
        # _RENPATH = FILEPATH.parent / (str(FILEPATH.stem) + "_renamed" + str(FILEPATH.suffix))
        _RENPATH = FILEPATH
        
        lins = f.readlines()
        _lins = lins.copy()

        for i,lin in enumerate(lins):
            _match = re.search(text_re, lin)
            if _match:
                for _mtch in re.finditer(text_re, lin):
                    if _mtch not in no_rep:
                        _new = abbr[FILEPATH.name] + "_" + _mtch.group(0)
                        _lins[i] = re.sub(_mtch.group(0), _new,_lins[i])

                        keys += [_mtch.group(0)]

    if len(keys) >0:
        texts[FILEPATH.name] = keys

        with open(_RENPATH, "w") as f:
            f.writelines(_lins)

In [100]:
# with open(RENPATH, "w") as ren:
with open(ALLVPATH, "r") as alv:
    lins = alv.readlines()
    _lins = lins.copy()
    for i, lin in enumerate(lins):
        if re.search("msgid", lin):
            for nam, plcs in texts.items():
                for plc in plcs:
                    if re.search(plc, lin):
                        _lins[i] = re.sub(plc, abbr[nam] + "_" + plc, lin)
                        break
                    
with open(ALLVPATH, "w") as f:
    f.writelines(_lins)       
                